In [ ]:
import torch
import torchvision

In [ ]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), 'model.pth')
      torch.save(optimizer.state_dict(), 'optimizer.pth')

In [ ]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
fig

In [ ]:
with torch.no_grad():
  output = network(example_data)

In [ ]:
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])
fig


In [ ]:
network.parameters()

In [ ]:
network

In [ ]:
network.state_dict()

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/me-mnist"

role = sagemaker.get_execution_role()

In [ ]:
role

In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3')

In [ ]:
bucket

In [ ]:
import tarfile
import os.path
output_filename = "model.tar.gz"
source_dir = "model.pth"
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [ ]:
make_tarfile(output_filename,source_dir)

In [ ]:
s3.meta.client.upload_file("model.tar.gz", bucket, 'me-mnist.tar.gz')

In [ ]:
for my_bucket_object in s3.Bucket(bucket).objects.all():
    print(my_bucket_object)

In [ ]:
model_s3_key = "me-mnist.tar.gz"
model_url = f's3://{bucket}/{model_s3_key}'  

In [ ]:
region = sagemaker_session.boto_region_name

In [ ]:
!pip install -U sagemaker

In [ ]:
from sagemaker import image_uris

# Name of the framework or algorithm
framework='pytorch'
#framework='xgboost' # Example

# Version of the framework or algorithm
version = '1.12.1'
#version = '0.90-1' # Example

# Specify an AWS container image. 
# container = image_uris.retrieve(region=region, 
#                                 framework=framework, 
#                                 version=version)
container = image_uris.retrieve(framework='pytorch',region='ap-south-1',version='1.8.0',py_version='py3',image_scope='inference', instance_type='ml.c5.4xlarge')

In [ ]:
container

In [ ]:
s3_output_location = bucket  + "/output"

In [ ]:
torch.__version__

In [ ]:
!cat "/opt/conda/lib/python3.8/site-packages/sagemaker/image_uri_config/PyTorch.json"

In [ ]:
!ls "/opt/conda/lib/python3.7/site-packages/sagemaker/image_uri_config"

In [ ]:
!cat "/opt/conda/lib/python3.7/site-packages/sagemaker/image_uri_config/pytorch.json"

In [ ]:
sagemaker_session.__dict__

In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
role

In [ ]:
pytorch_model = PyTorchModel(
    model_data=model_url, 
    role=role, 
    entry_point='inference.py', 
    framework_version='1.3.1',
    py_version="py3"
)

In [ ]:
pytorch_model

In [ ]:
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)

In [ ]:
predictor

In [ ]:
import random
import numpy as np

dummy_data = {"inputs": np.random.rand(16, 1, 28, 28).tolist()}

In [ ]:
dummy_data

In [ ]:
res = predictor.predict(dummy_data)

In [ ]:
model_url

In [ ]:
!cp model.tar.gz me-mnist.tar.gz

In [ ]:
!cp me-mnist.tar.gz /opt/ml/model/me-mnist.tar.gz

In [ ]:
!ls /opt/ml/model

In [ ]:
predictor.delete_endpoint()

In [ ]:
!ls -ltrh

In [ ]:
!ls -l /opt/ml/model

In [ ]:
!cat /opt/ml/model

In [ ]:
!rm /opt/ml/model

In [ ]:
torch.__version__

In [ ]:
pytorch_model.__dict__